In [1]:
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np


class BatchLoader():
    def __init__(self, data_dir, batch_size, seq_length):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length

        input_file = os.path.join(data_dir, "tangent_space_eeg.csv")
        
        self.preprocess(input_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file):
        self.tensor = np.genfromtxt(input_file, delimiter=",")
        self.tensor = self.tensor.reshape(len(self.tensor), 1, 12)

    def create_batches(self):
        self.num_batches = int(len(self.tensor) / (self.batch_size *
                                                   self.seq_length))
        print("num_batches: %s" % (self.num_batches))

        # When the data (tensor) is too small,
        # let's give them a better error message
        if self.num_batches == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        batched_data = self.tensor.reshape(-1, self.seq_length, 12)
        xdata = batched_data[:,:,2:12]
        ydata = batched_data[:,self.seq_length,0:2]
        self.x_batches = np.array_split(xdata,
                                  self.num_batches, 0)
        self.y_batches = np.array_split(ydata,
                                  self.num_batches, 0)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

In [3]:
from utils import BatchLoader

# Training Parameters
learning_rate = 0.001
epochs = 10000
batch_size = 3
display_step = 200

# Network Parameters
num_imput = 10 # Number of dimension in tangent space via pyriemann
timesteps = 5 # Five two-second eeg epochs per sequence
num_hidden = 64 # hidden layer num of features
num_classes = 2 # distracted or concentrated

X = []
y = []

# Initialize data feed
loader = BatchLoader('data', batch_size, timesteps) 
loader.reset_batch_pointer()
for i in range(loader.num_batches):
    batch_x, batch_y = loader.next_batch()
    X.append(batch_x)
    y.append(batch_y)

num_batches: 153


IndexError: index 5 is out of bounds for axis 1 with size 5

In [31]:
batch_y

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])